## Llama 2 Embeddings

In [10]:
from tqdm import tqdm
import json

In [11]:
import sys
sys.path.append('..')

from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from db_connect import db_get_df

model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [12]:
df = db_get_df(["*"], "transcript_segments")

In [26]:
type(df["segment_text"])

pandas.core.series.Series

In [13]:
input_ids = tokenizer(
    df["segment_text"].to_list(), 
    return_tensors = 'pt',
    padding = True
)

  0%|          | 0/2 [14:32:24<?, ?it/s]


In [14]:
v = model.encode(input_ids).detach().numpy()
v.shape

KeyboardInterrupt: 

In [8]:
df["embedding_json"] = [json.dumps(model.encode(chunk_text).detach().numpy()) for chunk_text in tqdm(input_ids)]


TypeError: forward() argument after ** must be a mapping, not str

In [16]:
input_ids["input_ids"][0]

tensor([    1,  1364,   276,  9268, 29892,  3056,  5243,  1620,  1011,   302,
        11129,   275,   303,  8403,   530,  3197,  1885, 29889,  1954, 26986,
         1425, 25434,   805,  5876,  3478,  2160,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0])

In [14]:
len(v[0])

1536